In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Flight Data Case Study").getOrCreate()

In [ ]:
airportDF = spark.read.load("airportdata.csv", format="csv", inferSchema="true",header="true")

In [ ]:
flightDF = spark.read.load("flightdata.csv", format="csv",inferSchema="true",header="true")

In [ ]:
# Check the count of records of the dataframe.
airportDF.count()

In [ ]:
flightDF.count()

In [ ]:
airportDF.printSchema()

In [ ]:
# Check the schema of the dataframe. Specifically look for airTime and arrivalDelay columns of flight data
flightDF.printSchema()

In [ ]:
airportDF.show(5,False)

In [ ]:
flightDF.show(5,False)

In [ ]:
# Check the data description i.e. stats of all columns
airportDF.describe().show()

In [ ]:
flightDF.describe().show()

In [ ]:
# For a specific column the stats can be checked as below
flightDF.describe("distance").show()

In [ ]:
from pyspark.sql.functions import *

In [ ]:
flightDF.where((col("airTime") != "NA")|(col("arrivalDelay") != "NA")).count()

In [ ]:
# 1. Check the schema of the data frame after having flight data. The columns airtime and arrivalDelay would be inferred as string.
# This is because some of the rows have “NA” in these columns. Remove these rows using filter function and cast these columns to integer.

flightDF2=flightDF.where((col("airTime") != "NA")|(col("arrivalDelay") != "NA")).withColumn("airTime",col("airTime").cast("Integer")).\
    withColumn("arrivalDelay",col("arrivalDelay").cast("Integer"))

In [ ]:
flightDF2.count()

In [ ]:
flightDF2.filter(col("arrivalDelay")=="NA").count()

In [ ]:
flightDF2.filter(col("airTime")=="NA").count()

In [ ]:
flightDF2.describe().show()

In [ ]:
# Now check and compare the data type of airTime and arrivalDelay columns
flightDF2.printSchema()

In [ ]:
# 2. As some of the reports require month-wise breakup of flight data, add a column with the number of the month and another column with the name of the month extracted from the date column.

# Add a column start_to_date by converting startDate column from string to date

flightDF3=flightDF2.withColumn("start_to_date",(to_date(col("startDate"), "M/d/yyyy")))

In [ ]:
flightDF3.printSchema()

In [ ]:
flightDF3.show()

In [ ]:
# Example date functions
flightDF3.select( "start_to_date",
                 year(col("start_to_date")).alias("year"),
                 month(col("start_to_date")).alias("month"),
                 dayofweek(col("start_to_date")).alias("dayofweek"), 
                 date_format("start_to_date", "EEEE").alias("dayofweek_full"),
                 date_format("start_to_date", "E").alias("dayofweek_short"),
                 dayofmonth(col("start_to_date")).alias("dayofmonth"), 
                 dayofyear(col("start_to_date")).alias("dayofyear"), 
                 next_day(col("start_to_date"),"Sunday").alias("next_day"), 
                 weekofyear(col("start_to_date")).alias("weekofyear"),
                 date_format("start_to_date", "LLLL").alias("monthname_full"),
                 date_format("start_to_date", "LLL").alias("monthname_short"),
                 date_format("start_to_date", "QQQ").alias("Quarter1"),
                 date_format("start_to_date", "q").alias("quarter2")
   ).show(30)

In [ ]:
flightDF4 = flightDF3.withColumn("month",(month(col("start_to_date"))))

In [ ]:
flightDF4.printSchema()

In [ ]:
flightDF4.show()

In [ ]:
flightDF4a = flightDF3.withColumn("monthname",date_format("start_to_date", "LLL"))

In [ ]:
flightDF4a.printSchema()

In [ ]:
flightDF4a.show(35)

In [ ]:
# Using when & otherwise function
flightDF5=flightDF4.withColumn("cancelReason", when( col("cancellationCode")==1,lit("carrier")).
                               when( col("cancellationCode")==2,lit("weather")).
                               when( col("cancellationCode")==3,lit("NAS")).
                               when( col("cancellationCode")==4,lit("security")).
                               otherwise(col("cancellationCode")) )

In [ ]:
flightDF5.printSchema()

In [ ]:
flightDF5.show()

In [ ]:
# Using a Map and .na.replace function contents of the existing column can be replaced with new values
# This will not add a new column. It uses the existing column and replaces the values
ccdict={1:"carrier", 2:"weather", 3:"NAS", 3:"security"}
flightDF5A=flightDF4.withColumn("cancellationCode",col("cancellationCode").cast("String"))\
    .na.replace(['1','2','3','4'], ['carrier','weather','NAS','security'],"cancellationCode")

In [ ]:
flightDF5A.printSchema()

In [ ]:
flightDF5A.show()

In [ ]:
airportDF.show(5)

In [ ]:
# 4. As some of the reports require airport names to be provided, add the columns of the airport data by join.
# Join is done with airport data on airport code and origin code once and the on airport code and destination code

flightDF6 = flightDF5.join(airportDF,flightDF5.origin == airportDF.code, "inner").\
withColumnRenamed("code","originCode").withColumnRenamed("name","originName").\
withColumnRenamed("areaCode","originAreaCode")

In [ ]:
flightDF6.printSchema()

In [ ]:
flightDF6.show(5,False)

In [ ]:
flightDF7 = flightDF6.join(airportDF,flightDF6.destination == airportDF.code, "inner").\
    withColumnRenamed("code","destCode").withColumnRenamed("name","destName").withColumnRenamed("areaCode","destAreaCode")

In [ ]:
flightDF7.printSchema()

In [ ]:
flightDF7.show(5,False)

In [ ]:
# 5. Give the breakup of number of cancellations based on origin airport. Which origin airport had maximum cancellations? Airport names should be mentioned in the list in addition to the airport code.
flightDF7.filter(col("cancelled")==1).count() # to verify total count

In [ ]:
flightDF7.filter(col("cancelled")==1).groupBy("originName").\
agg(count("cancelled").alias("cancel_count"))\
.orderBy(col("cancel_count").desc())\
.show(5,False)

In [ ]:
fDF8=flightDF7.filter(col("cancelled")==1).\
groupBy("originName").agg(count("cancelled").alias("cancel_count")).\
orderBy(col("cancel_count").desc())
# Save the result into a Hive table or a CSV file as required

In [ ]:
fDF8.write.mode("append").save("CancelledLists",format="csv",header=True)

In [ ]:
fDF8.show()

In [ ]:
# 6. Give reason-wise breakup of cancellations. What is the reason recorded for maximum number of cancellations?
flightDF7.filter(col("cancelled")==1).groupBy("cancelReason").\
agg(count("cancelled").alias("cancel_count")).\
orderBy(col("cancel_count").desc())\
.show(5,False)

In [ ]:
fDF9=flightDF7.filter(col("cancelled")==1).\
groupBy("cancelReason").agg(count("cancelled").alias("cancel_count")).\
orderBy(col("cancel_count").desc())
# Save the result into a Hive table or a CSV file as required

In [ ]:
fDF9.write.mode("append").save("CancelledLists",format="csv",header=True)

In [ ]:
fDF9.show()

In [ ]:
#7. Give month-wise breakup of cancellations from the available data. Which month has maximum number of cancellations?
flightDF7.filter(col("cancelled")==1).\
groupBy("month").agg(count("cancelled").alias("cancel_count")).\
orderBy(col("cancel_count").desc()).show()

In [ ]:
fDF10=flightDF7.filter(col("cancelled")==1).\
groupBy("month").agg(count("cancelled").alias("cancel_count")).\
orderBy(col("cancel_count").desc())
fDF10.show()

In [ ]:
# 8. Give the breakup of number of diversions based on destination airport. Which destination airport caused maximum diversions? Airport names should be mentioned in the list in addition to the airport code.
flightDF7.filter(col("diverted")=="Yes").count() # to verify total count

In [ ]:
flightDF7.filter(col("diverted")=="Yes").\
groupBy("destName").agg(count("diverted").alias("divert_count")).\
orderBy(col("divert_count").desc()).show(5,False)

In [ ]:
fDF11= flightDF7.filter(col("diverted")=="Yes").\
groupBy("destName").agg(count("diverted").alias("divert_count")).\
orderBy(col("divert_count").desc())
# Save the result into a Hive table or a CSV file as required
fDF11.show(5,False)

In [ ]:
# 9. Give month-wise breakup of diversions from the available data. Which month has maximum number of diversions?
flightDF7.filter(col("diverted")=="Yes").\
groupBy("month").agg(count("diverted").alias("divert_count")).\
orderBy(col("divert_count").desc()).show(truncate=False)

In [ ]:
fDF12=flightDF7.filter(col("diverted")=="Yes").\
groupBy("month").agg(count("diverted").alias("divert_count")).\
orderBy(col("divert_count").desc())
# Save the result into a Hive table or a CSV file as required
fDF12.show(truncate=False)

In [ ]:
flightDF7.filter(col("arrivalDelay")<0).count()

In [ ]:
flightDF7.filter(col("arrivalDelay")==0).count()

In [ ]:
flightDF7.filter(col("arrivalDelay")>0).count()

In [ ]:
fDF13=flightDF7.\
withColumn("arrivedAhead", when(col("arrivalDelay")<0,1 )).\
withColumn("arrivedOnTime", when(col("arrivalDelay")==0,1 )).\
withColumn("arrivedLate", when(col("arrivalDelay")>0,1 ))
fDF13.show()
fDF13.printSchema()

In [ ]:
fDF13A= fDF13.agg(\
    sum("arrivedAhead").alias("arrivedAheadCount"), \
    sum("arrivedOnTime").alias("arrivedOnTimeCount"), \
    sum("arrivedLate").alias("arrivedLateCount") \
)
fDF13A.show()
fDF13A.printSchema()

In [ ]:
fDF13B=flightDF7. \
withColumn("arrivedAhead", when(col("arrivalDelay")<0,1 )). \
withColumn("arrivedOnTime", when(col("arrivalDelay")==0,1 )). \
withColumn("arrivedLate", when(col("arrivalDelay")>0,1 )). \
agg(\
    sum("arrivedAhead").alias("arrivedAheadCount"), \
    sum("arrivedOnTime").alias("arrivedOnTimeCount"), \
    sum("arrivedLate").alias("arrivedLateCount") \
)
fDF13B.show(5,False)
fDF13B.printSchema()

In [ ]:
flightDF7. \
withColumn("arrivedAhead", when(col("arrivalDelay")<0,1 )). \
withColumn("arrivedOnTime", when(col("arrivalDelay")==0,1 )). \
withColumn("arrivedLate", when(col("arrivalDelay")>0,1 )). \
agg( \
    sum("arrivedAhead").alias("arrivedAheadCount"), \
    sum("arrivedOnTime").alias("arrivedOnTimeCount"), \
    sum("arrivedLate").alias("arrivedLateCount") \
).show()

In [ ]:
# 11. Generate the above list for flights that have been diverted.
fDF14=flightDF7.filter(col("diverted")=="Yes"). \
withColumn("arrivedAhead", when(col("arrivalDelay")<0,1 )). \
withColumn("arrivedOnTime", when(col("arrivalDelay")==0,1 )). \
withColumn("arrivedLate", when(col("arrivalDelay")>0,1 )). \
agg( \
    sum("arrivedAhead").alias("arrivedAheadCount"), \
    sum("arrivedOnTime").alias("arrivedOnTimeCount"), \
    sum("arrivedLate").alias("arrivedLateCount") \
)
fDF14.show(5,False)
fDF14.printSchema()

In [ ]:
# 12. Generate the above list for flights that have not been diverted.
fDF15=flightDF7.filter(col("diverted")=="No"). \
withColumn("arrivedAhead", when(col("arrivalDelay")<0,1 )). \
withColumn("arrivedOnTime", when(col("arrivalDelay")==0,1 )). \
withColumn("arrivedLate", when(col("arrivalDelay")>0,1 )). \
agg( \
    sum("arrivedAhead").alias("arrivedAheadCount"), \
    sum("arrivedOnTime").alias("arrivedOnTimeCount"), \
    sum("arrivedLate").alias("arrivedLateCount") \
)
fDF15.show(5,False)
fDF15.printSchema()


In [ ]:
spark.stop()